# 4 - Mismatch Equation calculation for fixed-tilt setup

This jupyter journal will walk us through the creation of a basic fixed-tilt simulation possible with bifacialvf, and then use PinPV equation to calculate electrical mismatch for each hour, for hte year, and the factor to use for PVSyst which is applied to Grear on their workflow.

Electrical mismatch calculation following Progress in PV paper
    Estimating and parameterizing mismatch power loss in bifacial photovoltaic systems
    Chris Deline, Silvana Ayala Pelaez,Sara MacAlpine,Carlos Olalla
    https://doi.org/10.1002/pip.3259 
    

THIS JOURNAL IS IN DEVELOPMENT


In [1]:
from pathlib import Path
import os
import bifacialvf
import bifacial_radiance as br
import pandas as pd
import numpy as np
# IO Files
testfolder = Path().resolve().parent.parent / 'bifacialvf' / 'TEMP' / 'Tutorial_03'
if not os.path.exists(testfolder):
    os.makedirs(testfolder)

In [2]:
# Download and Read input
TMYtoread=bifacialvf.getEPW(lat=37.5407,lon=-77.4360, path = testfolder)
myTMY3, meta = bifacialvf.readInputTMY(TMYtoread)
deltastyle = 'TMY3'  # 

path = C:\Users\sayala\Documents\GitHub\bifacialvf\bifacialvf\TEMP\Tutorial_03
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!


In [3]:
# Variables
config=5              # 5-up configuration
panellength = 1*config           #1 meter * 5 modules on 1-up landscape
tilt = 10                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg) or tracker axis direction
albedo = 0.62               # ground albedo
clearance_height=1/panellength   # 1m clearance normalized to panel length
pitch = 5/panellength       # row to row spacing in normalized panel lengths. 
rowType = "interior"        # RowType(first interior last single)
transFactor = 0.013         # TransmissionFactor(open area fraction, including x-gaps and y-gaps in the collector surface)
sensorsy = 6*5 # Number of sampling points for the setup. Recommend at least 6 per module. 
PVfrontSurface = "glass"    # PVfrontSurface(glass or ARglass)
PVbackSurface = "glass"     # PVbackSurface(glass or ARglass)

# Calculate PV Output Through Various Methods    
# This variables are advanced and explored in other tutorials.
bififactor = 0.7              # IF monofacial set to 0

# Tracking instructions
tracking=False
backtrack=False
limit_angle = 60


writefiletitle = os.path.join(testfolder, 'Tutorial3_Results.csv')
myTMY3 = myTMY3.iloc[0:24].copy()  # Simulate just the first 24 hours of the data file for speed on this example
bifacialvf.simulate(myTMY3, meta, writefiletitle=writefiletitle, 
         tilt=tilt, sazm=sazm, pitch=pitch, clearance_height=clearance_height, 
         rowType=rowType, transFactor=transFactor, sensorsy=sensorsy, 
         PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface, 
         albedo=albedo, tracking=tracking, backtrack=backtrack, 
         limit_angle=limit_angle, deltastyle=deltastyle)




Calculating Sun position with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Albedo value passed, but also present in TMY3 file.  Using albedo value passed. To use the ones in TMY3 file re-run simulation with albedo=None

 
********* 
Running Simulation for TMY3: 
Location:   RICHMOND
Lat:  37.5  Long:  -77.33  Tz  -5.0
Parameters: tilt:  10   Sazm:  180     Clearance_Height :  0.2   Pitch:  1.0   Row type:  interior   Albedo:  0.62
Saving into C:\Users\sayala\Documents\GitHub\bifacialvf\bifacialvf\TEMP\Tutorial_03\Tutorial3_Results.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  0.4510244972733475
Actual distance between rows =  0.01519224698779198
 


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 64.52it/s]

Finished


## 2. Load the irradiance results

In [4]:
from bifacialvf import loadVFresults
mismatchResultstitle = os.path.join(testfolder, 'Tutorial3_Results.csv')
(data, metadata) = loadVFresults(mismatchResultstitle)


In [5]:
data.keys()

Index(['date', 'DNI', 'DHI', 'albedo', 'decHRs', 'ghi', 'inc', 'zen', 'azm',
       'pvFrontSH', 'aveFrontGroundGHI', 'GTIfrontBroadBand', 'pvBackSH',
       'aveBackGroundGHI', 'GTIbackBroadBand', 'maxShadow', 'Tamb', 'VWind',
       'No_1_RowFrontGTI', 'No_2_RowFrontGTI', 'No_3_RowFrontGTI',
       'No_4_RowFrontGTI', 'No_5_RowFrontGTI', 'No_6_RowFrontGTI',
       'No_7_RowFrontGTI', 'No_8_RowFrontGTI', 'No_9_RowFrontGTI',
       'No_10_RowFrontGTI', 'No_11_RowFrontGTI', 'No_12_RowFrontGTI',
       'No_13_RowFrontGTI', 'No_14_RowFrontGTI', 'No_15_RowFrontGTI',
       'No_16_RowFrontGTI', 'No_17_RowFrontGTI', 'No_18_RowFrontGTI',
       'No_19_RowFrontGTI', 'No_20_RowFrontGTI', 'No_21_RowFrontGTI',
       'No_22_RowFrontGTI', 'No_23_RowFrontGTI', 'No_24_RowFrontGTI',
       'No_25_RowFrontGTI', 'No_26_RowFrontGTI', 'No_27_RowFrontGTI',
       'No_28_RowFrontGTI', 'No_29_RowFrontGTI', 'No_30_RowFrontGTI',
       'No_1_RowBackGTI', 'No_2_RowBackGTI', 'No_3_RowBackGTI',
       'No_4_RowB

In [6]:
df = data[(list(data.filter(regex='RowFrontGTI')))]
db = data[(list(data.filter(regex='RowBackGTI')))]

# Stripping names so that I can easily add them
df.columns = [col.strip('_RowFrontGTI') for col in df.columns]
db.columns = [col.strip('_RowBackGTI') for col in db.columns]


## 3.  Calculate the Panel Irradiance Input (Front + rear * bifaciality Factor)

In [7]:
irradiances = df+db*bififactor
irradiances

,No_1,No_2,No_3,No_4,No_5,No_6,No_7,No_8,No_9,No_10,...,No_21,No_22,No_23,No_24,No_25,No_26,No_27,No_28,No_29,No_30
0,7.026827,8.317857,9.456788,10.421661,11.208600,11.748774,12.157833,12.525715,12.731163,12.920716,...,13.772289,16.372518,16.392185,16.389620,16.403242,16.407244,16.420732,16.423879,16.442330,16.470671
1,27.672987,32.706288,37.146515,40.908430,43.976304,46.095734,47.690263,49.124590,49.925224,50.663842,...,125.813391,125.999091,126.156134,126.138973,126.259015,126.292357,126.395140,126.387919,126.470950,126.612308
2,42.668564,50.516803,57.440432,63.305918,68.089773,71.346397,73.833112,76.069470,80.260734,83.391109,...,87.751418,87.714539,87.666803,87.651943,87.586275,87.554837,87.507019,87.528174,87.574563,87.635573
3,72.009677,85.245617,96.922199,106.814289,114.882165,120.376947,124.570709,142.094804,150.076467,152.019795,...,159.398431,159.353207,159.287849,159.261508,159.163421,159.113836,159.042776,159.074966,159.155499,159.264437
4,84.794592,100.381497,114.132072,125.781190,135.282083,141.754078,149.810933,172.646928,175.127348,177.415859,...,186.118382,186.073636,186.004345,185.973495,185.864929,185.809367,185.732006,185.770375,185.868844,186.002914
5,78.152469,92.518983,105.192946,115.929983,124.686997,130.651680,139.351095,158.481421,160.767638,162.876972,...,170.892052,170.846837,170.779402,170.751037,170.647840,170.595511,170.521535,170.557091,170.646611,170.767990
6,81.106698,96.013390,109.163887,120.304646,129.390914,135.580145,140.303272,167.052410,170.566540,172.755162,...,181.073959,181.028812,180.960392,180.930546,180.824581,180.770257,180.693974,180.729738,180.822004,180.947624
7,75.452654,89.317157,101.548241,111.910115,120.361115,126.117554,130.510452,139.701566,162.722583,164.758165,...,172.493922,172.451260,172.386976,172.358774,172.259356,172.208023,172.135775,172.167836,172.252056,172.366986
8,40.890027,48.409119,55.042374,60.661876,65.245095,68.365238,70.747652,72.890223,74.086790,77.696758,...,86.687997,86.653250,86.608013,86.593486,86.530827,86.500490,86.454548,86.474024,86.517794,86.575619
9,12.303996,14.567793,16.564890,18.256767,19.636655,20.575574,21.292862,21.937930,22.298193,22.630584,...,24.574212,24.560571,24.544106,24.539908,24.518623,24.508757,24.493008,24.499349,24.511898,24.527954


## 4. Calculate a sample module performance to know the yearly derate. 
This is a proxy for the collector 

In [8]:
#CEC Parameters for the modules desired can be used here, and can be found through SAM, pvlib, or the California CEC database.

CECMod_longi_df = pd.DataFrame({'alpha_sc': 0.0038991, 'a_ref': 1.78308, 'I_L_ref': 9.51892,'I_o_ref': 2.03E-11, 'R_sh_ref': 411.557, 'R_s': 0.386111,'Adjust': 7.35293}, index=[0])


In [9]:
# Calculate a reference power using bifacial radiance internal methods
# This corrects for Temperature and Wind
power_ref = br.performance.calculatePerformance(
    irradiances.sum(axis=1),temp_air=data['Tamb'],wind_speed=data['VWind'],
    CECMod=CECMod_longi_df)


## 5. Calculate Hourly Mismatch with bifacial radiance internal equation fit

In [10]:
hourlymismatch=br.mismatch.mismatch_fit3(irradiances.T)/100
#Clipping mismatch for edge cases
hourlymismatch[hourlymismatch>5]=5

c:\users\sayala\documents\github\bifacial_radiance\bifacial_radiance\mismatch.py:220: FutureWarning: outer method for ufunc <ufunc 'subtract'> is not implemented on pandas objects. Returning an ndarray, but in the future this will raise a 'NotImplementedError'. Consider explicitly converting the DataFrame to an array with '.to_numpy()' first.
  return (np.abs(np.subtract.outer(data,data)).sum()/float(data.__len__())**2 / np.mean(data))*100


In [11]:
# Calculate power reduction
powerreduced = power_ref*(100-hourlymismatch)/100

# This is the Yearly Mismatch then:
YEARLYMismatch = (power_ref.sum()-powerreduced.sum())*100/power_ref.sum()

## 6. Convert the Yearly Mismatch to the Mismatch Loss Factor from PVSyst

Follows the same journal procedure. ANotehr visual explanation on the slides and poster for the same journal:

    https://www.nrel.gov/docs/fy19osti/74831.pdf
    https://www.nrel.gov/docs/fy19osti/74236.pdf

In [12]:
# Need to calculate hte bifacial gain first:
BifacialGain = db.mean(axis=1).sum()*bififactor*100/df.mean(axis=1).sum()
BifacialGain

PVSyst_Mismatch = YEARLYMismatch/BifacialGain + YEARLYMismatch
print("PVSyst Mismatch applied to Grear", np.round(PVSyst_Mismatch,2))

PVSyst Mismatch applied to Grear 3.43
